# What's Cooking  in Python
https://www.kaggle.com/manuelatadvice/whats-cooking/noname/code

In [ ]:
#You might need to install this. 
#!conda install nltk

In [ ]:
#This imports a bunch of packages.  
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from collections import Counter
import json
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
#from sklearn import grid_search



In [ ]:

#If you import the codes locally, this seems to cause some issues.  
import json
from urllib.request import urlopen

urltrain= 'https://raw.githubusercontent.com/RPI-Analytics/MGMT6963-2015/master/data/whatscooking/whatscookingtrain.json'
urltest = 'https://raw.githubusercontent.com/RPI-Analytics/MGMT6963-2015/master/data/whatscooking/whatscookingtest.json'


train = pd.read_json(urlopen(urltrain))
test = pd.read_json(urlopen(urltest))

In [ ]:
#First we want to see the most popular cuisine for the naive model. 
train.groupby('cuisine').size()

In [ ]:
#Here we write the most popular selection.  This is the baseline by which we will judge other models. 
test['cuisine']='italian'

In [ ]:
#THis is a much more simple version that selects out the columns ID and cuisinte
submission=test[['id' ,  'cuisine' ]]
#This is a more complex method I showed that gives same.
#submission=pd.DataFrame(test.ix[:,['id' ,  'cuisine' ]])

In [ ]:
#This outputs the file.
submission.to_csv("1_cookingSubmission.csv",index=False)


In [ ]:
#So it seems there is some data we need to use the NLTK leemmatizer.  
stemmer = WordNetLemmatizer()
nltk.download('wordnet')

 

In [ ]:
train

In [ ]:
#We see this in a Python Solution. 
train['ingredients_clean_string1'] = [','.join(z).strip() for z in train['ingredients']] 

#We also know that we can do something similar though a Lambda function. 
strip = lambda x: ' , '.join(x).strip() 
#Finally, we call the function for name
train['ingredients_clean_string2'] = train['ingredients'].map(strip)

#Now that we used the lambda function, we can reuse this for the test dataset. 
test['ingredients_clean_string1'] = test['ingredients'].map(strip)
 


In [ ]:
#We see this in one of the solutions.  We can reconstruct it in a way that makes it abit easier to follow, but I found when doing that it took forever.  

#To interpret this, read from right to left. 
train['ingredients_string1'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in train['ingredients']]       
test['ingredients_string1'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in test['ingredients']]       




In [ ]:
train['ingredients_string1']

In [ ]:
ingredients = train['ingredients'].apply(lambda x:','.join(x))
ingredients

In [ ]:
#Now we will create a corpus.
corpustr = train['ingredients_string1']
corpusts = test['ingredients_string1']
corpustr

In [ ]:
#http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
#You could develop an understanding based on each.  
vectorizertr = TfidfVectorizer(stop_words='english',
                             ngram_range = ( 1 , 1 ),analyzer="word", 
                             max_df = .57 , binary=False , token_pattern=r'\w+' , sublinear_tf=False)
vectorizerts = TfidfVectorizer(stop_words='english')

In [ ]:
#Note that this doesn't work with the #todense option.  
tfidftr=vectorizertr.fit_transform(corpustr)
predictors_tr = tfidftr

In [ ]:
#Note that this doesn't work with the #todense option.  This creates a matrix of predictors from the corpus. 
tfidfts=vectorizertr.transform(corpusts)
predictors_ts= tfidfts

In [ ]:
#This is target variable.  
targets_tr = train['cuisine']


## Logistic Regression and Regularization.

- Regularlization can help us with the large matrix by adding a penalty for each parameter. 
- Finding out how much regularization via grid search (search across hyperparameters.)

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

```C : float, default: 1.0
Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.```

In [ ]:
#Logistic Regression. 
parameters = {'C':[1, 10]}
#clf = LinearSVC()
clf = LogisticRegression()



In [ ]:
predictors_tr

In [ ]:
from sklearn.model_selection import GridSearchCV
#This uses that associated paramters to search a grid space. 
classifier = GridSearchCV(clf, parameters)
classifier=classifier.fit(predictors_tr,targets_tr)



In [ ]:
#This predicts the outcome for the test set. 
predictions=classifier.predict(predictors_ts)

In [ ]:
#This adds it to the resulting dataframe. 
test['cuisine'] = predictions

In [ ]:
#This creates the submision dataframe
submission2=test[['id' ,  'cuisine' ]]

In [ ]:
#This outputs the file.
submission2.to_csv("2_logisticSubmission.csv",index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier 



In [ ]:
# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 10)



In [ ]:
# Fit the training data to the Survived labels and create the decision trees
forest = forest.fit(predictors_tr,targets_tr)



In [ ]:
# Take the same decision trees and run it on the test data
predictions = forest.predict(predictors_ts)

In [ ]:
#This adds it to the resulting dataframe. 
test['cuisine'] = predictions

In [ ]:
#This creates the submision dataframe
submission2=test[['id' ,  'cuisine' ]]

In [ ]:
ingredients = train['ingredients'].apply(lambda x:','.join(x))
ingredients
train

In [ ]:
#What we really need is a ingredient freqency, inverse cuisine frequency.  
#Let's first create a function to create a count
def tf(count, N):
        return count / float(N)

#Now let's create a function for IDF....
def idf(count, N ):

    # tf-idf calc involves multiplying against a tf value less than 0, so it's
    # necessary to return a value greater than 1 for consistent scoring. 
    # (Multiplying two values less than 1 returns a value less than each of 
    # them.)

    try:
        return 1.0 + log(float(N) / count)
    except ZeroDivisionError:
        return 1.0
    
    

In [ ]:
from collections import Counter

#This creates a Corpus for each incredient
corpus=','.join(x.strip() for x in train['ingredients_clean_string1']).split(',')
total=Counter(corpus)
#This creates a corpus for each word
corpus2=','.join(x.strip() for x in train['ingredients_string1']).split(' ')
#This just does some extra cleaning....because of some issues with file. 
corpus3=','.join(x.strip() for x in corpus2).split(',')
total2=Counter(corpus3)


In [ ]:
print(len(train))
countcuisines=Counter(train['cuisine'])
countcuisines

In [ ]:
# Now we want to create a function that can take in a dictionary [a lookup that has terms and values] and 
# score different recipes by the lookup.

def score(dictionary, stringx, delimiter):
    sumt=sum((dictionary.get(x,0) for x in stringx.split(delimiter)))
    return  sumt/len(stringx.split(delimiter))

#Alt.   
#    sumt=0
#    for x in stringx.split(delimiter):
#        sumt+=dictionary[x]
    
testdic={'salt': .5, 'vanilla': .2, 'butter': .3,'sugar': .1}
teststring='salt,vanilla,sugar,stink'
teststring2='salt vanilla sugar butter stink'
print(score(testdic,teststring,','))
print(score(testdic,teststring2,' '))
#print(score(testdic,teststring,' '))
print("Test1 Value: ", (.5+.2+.1)/4)
print("Test2 Value: ", (.5+.2+.1+.3)/5)


#Now we are using that separate function in another function.  
#title_fn = lambda x: 1 if has_title(x) else 0
#Finally, we call the function for name
#train['Title'] = train['Name'].map(title_fn)

In [ ]:
from math import log
trainc = pd.DataFrame()
trainc2 = pd.DataFrame()
testc = pd.DataFrame()
testc2 = pd.DataFrame()

#This will loop through each unique cuisine
for cuisine in train['cuisine'].unique():
#This selects only rows that are greek.

#Number of rows in the cuisine
    cuisinerows=train[train['cuisine'] == cuisine]
    notcuisinerows=train[train['cuisine'] != cuisine]
    
    #print (cuisinerows)
    
    #','.join(x.strip() for x in line.split(':'))
    
    #This looks at specific ingredients in the cuisine corpus
    cuisinecorpus=','.join(x.strip() for x in cuisinerows['ingredients_clean_string1']).split(',')
    #This looks at specific ingredients not in cuisine
    notcuisinecorpus=','.join(x.strip() for x in notcuisinerows['ingredients_clean_string1']).split(',')
   
    #This treats all words individually in cuisine
    cuisinecorpus2=','.join(x.strip() for x in cuisinerows['ingredients_string1']).split(' ')
    notcuisinecorpus2=','.join(x.strip() for x in notcuisinerows['ingredients_string1']).split(' ')
    #this extra line is just some additional cuisine
    cuisinecorpus2=','.join(x.strip() for x in cuisinecorpus2).split(',')
    notcuisinecorpus2=','.join(x.strip() for x in cuisinecorpus2).split(',')
    
    #This creates the document term matrix for each.
    tfcuisine=Counter(cuisinecorpus)
    tfnotcuisine=Counter(notcuisinecorpus)
    
    tfcuisine2=Counter(cuisinecorpus2)
    tfnotcuisine2=Counter(notcuisinecorpus2)
     
    #We didn't talk about it much in class, but this is creating a dict that indicates TFIDF for each ingredient
  
    cfincf={k: (tf(tfcuisine[k],len(cuisinecorpus))*idf(tfnotcuisine[k],len(notcuisinecorpus)))  for k in  tfcuisine.keys()}    
    
    #We didn't talk about it much in class, but this is creating a dict that indicates TFIDF for each word
    cfincf2={k: (tf(tfcuisine2[k],len(cuisinecorpus2))*idf(tfnotcuisine2[k],len(notcuisinecorpus2)))  for k in  tfcuisine2.keys()}    
    
    #Now we will use our strings to score each outcome. 
    score_fn = lambda x: score(cfincf, x, ',') 
    score_fn2 = lambda x: score(cfincf2, x, ' ')
    
#Finally, we call the function for name
    trainc[cuisine] = train['ingredients_clean_string1'].map(score_fn)
    trainc2[cuisine] = train['ingredients_clean_string1'].map(score_fn2)
    testc[cuisine] = test['ingredients_clean_string1'].map(score_fn)
    testc2[cuisine] = test['ingredients_clean_string1'].map(score_fn2)
    
    

In [ ]:
trainc 

In [ ]:

#This creates a prediction from the column that has the maximum value and outputs it to a file. 
test['cuisine']=testc.idxmax(axis=1)
submission=test[['id' ,  'cuisine' ]]
submission.to_csv("4_tfidf1.csv",index=False)
test['cuisine']=testc2.idxmax(axis=1)
submission=test[['id' ,  'cuisine' ]]
submission.to_csv("5_tfidf2.csv",index=False)


In [ ]:
trainc['prediction']=trainc.idxmax(axis=1)
trainc2['prediction']=trainc2.idxmax(axis=1)
trainc['cuisine']=train['cuisine']
trainc2['cuisine']=train['cuisine']
trainc.to_csv("coded1.csv",index=False)
trainc2.to_csv("coded2.csv",index=False)